In [7]:
import openai
import os
from openai.error import (RateLimitError, AuthenticationError, APIError, APIConnectionError, OpenAIError)
from openai.error import RateLimitError, InvalidRequestError, APIError, Timeout
import csv
import re
import time
import random
from dotenv import load_dotenv
import json

In [8]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [9]:
perspectives_dict = {
    'INFORMATION':  {'anchor-text': "For information purposes",
                    'tone': "Informative, Educational",
                    'definition': "Defined as knowledge about diseases, disorders, and health-related facts, providing insights into symptoms and diagnosis."
                    },
    'CAUSE':        {'anchor-text': "Some of the causes",
                    'tone': "Explanatory, Causal",
                    'definition': "Defined as reasons responsible for the occurrence of a particular medical condition, symptom, or disease"
                    },
    'SUGGESTION':   {'anchor-text': "It is suggested",
                    'tone': "Advisory, Recommending",
                    'definition': "Defined as advice or recommendations to assist users in making informed medical decisions, solving problems, or improving health issues."
                    },
    'EXPERIENCE':   {'anchor-text': "In user’s experience",
                    'tone': "Personal, Narrative",
                    'definition': "Defined as individual experiences, anecdotes, or firsthand insights related to health, medical treatments, medication usage, and coping strategies."
                    },
    'QUESTION':     {'anchor-text': "It is inquired",
                    'tone': "Seeking Understanding",
                    'definition': "Defined as inquiry made for deeper understanding."
                    },
    }

In [10]:
def get_prompt(data_sample, perspectives_dict):
    perspective = data_sample['perspective']
    spans = data_sample['spans']
    span_summary = data_sample['span_summary']
    question = data_sample['question']
    instruction = f"""Summarize the following content according to Perspective: {perspective};
{perspective} Definition: {perspectives_dict[perspective]['definition']}
Begin Summary with: {perspectives_dict[perspective]['anchor-text']}
Tone of summary: {perspectives_dict[perspective]['tone']}
Content to summarize: {", ".join(spans)}
Associated question: {question}"""
    output_text = span_summary

    return instruction, output_text


In [11]:
def generate(user_prompt):
    system_prompt = "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request."
    try:
    # Define the model and messages
        response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
        ],
        max_tokens = 2500,
        temperature = 0
        )

        content = response.choices[0].message["content"]
        print(content)
        return content

    except AuthenticationError:
        print("Error: Authentication failed. Please check your API key and try again.")
        time.sleep(10)
    except RateLimitError:
        print("Error: Rate limit exceeded. Please wait and try again later.")
        time.sleep(10)
    except InvalidRequestError as e:
        print(f"Error: Invalid request. Details: {str(e)}")
        time.sleep(10)
    except APIConnectionError:
        print("Error: Failed to connect to the API. Please check your network connection and try again")
        time.sleep(10)
    except Timeout:
        print("Error: The request timed out. Please try again later.")
        time.sleep(10)
    except OpenAIError as e:
        print(f"Error: An unexpected error occurred. Details: {str(e)}")
        time.sleep(10)
    except Exception as e:
        print(f"Error: An unknown error occurred. Details: {str(e)}")
        time.sleep(10)

In [ ]:
with open('../Dataset/InstructionSet/preprocessed_test.json', "r", encoding="utf-8") as f:
  dataset = json.load(f)

results = []
for sample in dataset:
  user_prompt, ref = get_prompt(sample, perspectives_dict)
  generated_summary = generate(user_prompt)
  if generated_summary:
    results.append({'uri': sample['uri'], 'out': generated_summary, 'ref': ref})
  
with open("./predictions_chatgpt.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

For information purposes, an orgasm, or sexual climax, is a pleasurable response to prolonged sexual stimulation, characterized by physiological reactions such as ejaculation in males and various responses in females. It represents the third stage of the human sexual response cycle, which includes excitement, plateau, orgasm, and resolution. The experience of orgasm is influenced by a combination of physical stimulation, emotional connection, and intellectual intimacy, often described as a peak of sensual and spiritual fulfillment. Factors such as thoughts about procreation, dynamics of dominance and submission, and the presence of true love can enhance the experience, although the occurrence of orgasms alone does not determine the longevity of a relationship.
For information purposes, synchronous menstruation refers to the phenomenon where women living or working closely together experience their menstrual cycles aligning over time. Observations have shown that this synchronization ca